In [2]:
import os
os.chdir("/Users/minhyeong-gyu/Documents/GitHub/quantum_kernel_investigation/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Quantum_Kernel import *

%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit , transpile, Aer ,IBMQ
from qiskit.visualization import *
from qiskit.tools.jupyter import *
from qiskit.providers.aer import QasmSimulator

import warnings
warnings.filterwarnings("ignore")

# Loading your IBM Quantum account(s)
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-kqc', group='internal', project='default')

In [54]:
mat = pd.read_table("data/yeast.data",header=None)
mat = pd.DataFrame([i.split() for i in mat[0]])
mat_x = mat.iloc[:,1:-1]
mat_y = mat.iloc[:,-1:]
mat_x = mat_x.applymap(float)
mat_xs = np.pi*(mat_x-np.min(mat_x))/(np.max(mat_x)-np.min(mat_x))
mat_ys = mat_y

In [55]:
data = pd.DataFrame(mat_xs)
data_y_orig = pd.DataFrame(mat_ys)
sample = 100
test_sample = 100

sto_ind = np.random.randint(0,len(data),sample)
data_sto = pd.DataFrame(data).iloc[sto_ind,:]
data_sto_y =pd.DataFrame(data_y_orig).iloc[sto_ind,:]
  
sto_ind = np.random.randint(0,len(data),test_sample)
data_sto_test = pd.DataFrame(data).iloc[sto_ind,:]
data_sto_test_y =pd.DataFrame(data_y_orig).iloc[sto_ind,:]

In [59]:
kernel_list =  [kernel_A,kernel_B,kernel_C,kernel_D,kernel_E,kernel_F]

In [60]:
gram_kernel_set= []
gram_kernel_test_set =[]
comp=1
backend = QasmSimulator()

gram_matrix = Gram_gaussian(data_sto,comp)
gram_test_matrix = Gram_gaussian_test(data_sto,data_sto_test,comp)
    
gram_kernel_set += [gram_matrix]
gram_kernel_test_set += [gram_test_matrix]

for i in range(len(kernel_list)) :
    gram_matrix = get_gram_multi(data=data_sto,kernel_fun=kernel_list[i],layer=1,backend=backend,shots=1000)
    gram_test_matrix = get_gram_test_multi(data=data_sto,test_data = data_sto_test,kernel_fun=kernel_list[i],layer=1,backend=backend,shots=1000)
    
    gram_kernel_set += [gram_matrix]
    gram_kernel_test_set += [gram_test_matrix]

100%|██████████| 100/100 [04:35<00:00,  2.75s/it]


In [62]:
save_folder = "result/yeast1"

pd.DataFrame(mat_xs).to_csv(save_folder+"/datasets_X_S.csv")
pd.DataFrame(mat_ys).to_csv(save_folder+"/datasets_y.csv")
pd.DataFrame(data_sto.index).to_csv(save_folder+"/sto_index.csv")
pd.DataFrame(data_sto_test.index).to_csv(save_folder+"/sto_test_index.csv")
for i in range(7):
   pd.DataFrame(gram_kernel_set[i]).to_csv(save_folder+"/gram_kernel"+str(i)+".csv")
   pd.DataFrame(gram_kernel_test_set[i]).to_csv(save_folder+"/gram_kernel_test"+str(i)+".csv")